# Lab 2: Stochastic Gradient Descent

The goal of this lab session is to code an optimization algorithm that optimzes the penalized loss function of the logistic regression model.

You have to send the filled notebook named **"L2_familyname1_familyname2.ipynb"** by email to *violeta.roizman@l2s.centralesupelec.fr* by October 10, 2018. Please put **"AML-L2"** in the subject. 

We begin with the standard imports:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
import math
from random import randint

We are going to use the W8A dataset, a tidy and binarized version of https://archive.ics.uci.edu/ml/datasets/adult (check it out for more details). 
In this dataset we have census data to predict if the income of an adult exceeds $50K/yr (1 or -1).  

In [ ]:
w8a_train = pd.read_csv("w8a.csv", sep=";")
w8a_train_x = w8a_train.iloc[:, :-1].values
w8a_train_y = w8a_train.iloc[:, -1].values

w8a_test  = pd.read_csv("w8a_t.csv", sep=";")
w8a_test_x = w8a_test.iloc[:, :-1].values
w8a_test_y = w8a_test.iloc[:, -1].values

print("shape: ", w8a_train.shape)
# w8a_train_x.head()

## Logistic Regression



Today we’ll be moving from linear regression to logistic regression, one of the simplest ways to deal with a classification problem. Instead of fitting a line, logistic regression models the probability that the outcome is 1 given the value of the predictor. In order to do this we need a function that transforms our predictor variable to a value between 0 and 1. Lots of functions can do that, but the logistic function is the most common choice:

$$f(z) = \frac{1}{1+\exp{-z}}.$$

To predict the class of our observations we'll have to minimize the corresponding loss function and as we are in a high-dimensional context we'll add an $l_2$ regularization to the model:

$$L(\textbf{w}) = \sum_{i=1}^n log(1+\exp(-y_i\textbf{w}^Tx_i))+\frac{\lambda}{2} \| \textbf{w} \|^2,$$

where $x_i$ is the vector of features for the observation $i$ and $y_i \in \{-1, 1\}$ is the class label.  


We first use the `sklearn` implementation:

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty="l2", C=1.0)
model.fit(w8a_train_x, w8a_train_y)
y_pred = model.predict(w8a_test_x)

and we compute the accuracy score to evaluate the model performance:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(w8a_test_y, y_pred)

### Assignment

Implement from scratch your own logistic regression model with stochastic gradient descent. 

- Fill in the classes

- Display the evolution of the cost function along iterations. Do this for several strategies for the setting of the learning rate

- Try the different acceleration strategies

- Train the model with the training set and evaluate its performance in the test set

In [ ]:
class LogisticRegression():
    """ Class for logistic regression:
    
    Attributes:
    -----------
    coef_: 1-dimensional np.array
        coefficients 
    alpha_: float
        regularization parameter
    bsize: integer
        size of the mini-batch
    averaging: boolean
        if true averaging is used, if not it isn't
    acceleration: string
        name of the aceleration strategy
    lr: float
        the learning rate
    max_iter:
        maximum number of iterations of the optimization algorithm
    
    """
    
    def __init__(self, alpha, bsize, averaging, acceleration, lr, max_iter):
        self.coef_  = None
        self.alpha_ = alpha
        self.bsize  = bsize
        self.averaging = averaging
        self.acceleration  = acceleration
        self.lr     = lr
        self.max_iter = max_iter
        #Loss function values
        self.l_f = None
        
    def fit(self, X, y):
        """ Fit the data (X, y).
    
        Parameters:
        -----------
        X: (num_samples, num_features) np.array
            Design matrix
        y: (num_sampes, ) np.array
            Output vector
        
        Note:
        -----
        Updates self.coef_
        """
        # TODO
        
        def f_lr(beta):            
            """ 
            Returns the logistic loss.        
            """
            
            logistic_loss = 0
            
            for i in range(len(y)):
                
                tmp = -y[i]*beta.T.dot(X[i, :])
                
                if(tmp > 10):
                    logistic_loss += + tmp
                else:
                    logistic_loss += np.log(1 + np.exp(tmp))
                                
            logistic_loss += self.alpha_*np.linalg.norm(beta)**2
                        
            return logistic_loss
        
        def fprime_lr(beta):   
            """ Returns the gradient of f_ls at beta.
            """
            
            # grad(F) = L.T * eˆ(L*beta) / (1 + eˆ(L*beta))
            
            # gradient descent
            if(self.bsize == len(y)):
                L = -np.diag(y).dot(X)
                tmp = L.dot(beta)
                grad = L.T.dot(np.exp(tmp)/(1 + np.exp(tmp))) + self.alpha_*beta
            
            # stochastic gradient descent (mini-batch)
            elif(self.bsize > 1 and self.bsize < len(y)):
                i_t = randint(0, np.rint((len(y)-1)/self.bsize))
                vec = range(self.bsize*(i_t-1)+1, self.bsize*i_t+1)
                L = np.zeros((self.bsize, X.shape[1]))
                t = 0
                for i in vec:
                    L[t, :] = -y[i]*X[i, :]
                    t += 1
                tmp = np.array(L.dot(beta))
                grad = L.T.dot(np.exp(tmp)/(1 + np.exp(tmp))) + self.alpha_*beta*self.bsize/X.shape[0]
            
            # stochastic gradient descent
            elif(self.bsize == 1):
                i_t = randint(0, len(y)-1)
                L = np.array(-y[i_t]*X[i_t, :])
                tmp = L.dot(beta)
                grad = L.T*np.exp(tmp)/(1 + np.exp(tmp)) + self.alpha_*beta/X.shape[0]   
            
            return grad
        
        
        def stochastic_gradient_descent():
            
            coef = np.zeros((X.shape[1])) 
                        
            loss_value = np.zeros((self.max_iter+1))
                        
            loss_value[0] = f_lr(coef)
            
            # vector that contains betaˆ(t-1) for the momentum variant
            tmp = np.zeros((X.shape[1]))
            
            if(self.averaging):
                mean = np.zeros((X.shape[1])) 
            
            for i in range(self.max_iter):
                
                #print(i)
                
                if(self.averaging):
                    self.lr = np.sqrt(1/(i+1)) 
                else:
                    self.lr = 1/(i+1) 
                
                # parameter for the momentum variant 
                theta = 0.9
                
                if(self.acceleration == 'momentum'):
                    # theta_t * (betaˆ(t) - betaˆ(t-1)) 
                    tmp1 = theta*(coef - tmp)
                    tmp = np.array(coef) 
                    # beta^(t+1) = betaˆ(t) - gamma_t * grad(F) + theta_t * (betaˆ(t) - betaˆ(t-1))
                    coef -= (self.lr*fprime_lr(coef) - tmp1)
                else:
                    # beta^(t+1) = betaˆ(t) - gamma_t * grad(F)
                    coef -= self.lr*fprime_lr(coef)
                    
                if(self.averaging):
                    # beta_ˆ(t) = (1 - 1/t) * beta_ˆ(t-1) + 1/t * betaˆ(t) 
                    mean = (1 - 1/(i+1))*mean + (1/(i+1))*coef
                    loss_value[i+1] = f_lr(mean)
                else:    
                    loss_value[i+1] = f_lr(coef)
                
                #print(loss_value[i+1])
                                                    
            if(self.averaging):
                self.coef_ = mean
            else:
                self.coef_ = coef
                                    
            plt.plot(range(i+1), loss_value[range(i+1)])
            self.l_f = loss_value
        stochastic_gradient_descent()
        
        
        
    
    def predict(self, X):
        """ Make binary predictions for data X.
    
        Parameters:
        -----------
        X: (num_samples, num_features) np.array
            Design matrix
        
        Returns:
        -----
        y_pred: (num_samples, ) np.array
            Predictions (0 or 1)
        """
        # TODO
        
        temp_pred = np.zeros(X.shape[0]) 
        
        for i in range(X.shape[0]):
            temp_pred[i] = np.sign(X[i, :].dot(self.coef_))
            #Processing the zero values
            if (temp_pred[i] == 0):
                temp_pred[i] = np.random.choice([-1,1])
        
        return temp_pred

Apply to the data

In [ ]:
num_it = 30
n_feat = w8a_train_x.shape[1]
lam = 20

"stochastic gradient descent"
dim_batch = 1
m1 = LogisticRegression(lam, dim_batch, False, 'l', 1e-2, num_it)
m1.fit(w8a_train_x, w8a_train_y)
p1 = m1.predict(w8a_test_x)
print("Accuracy:" + " "+str(accuracy_score(w8a_test_y, p1)))

"stochastic gradient descent - avec avaraging"
m2 = LogisticRegression(lam, dim_batch, True, 'l', 1e-2, num_it)
m2.fit(w8a_train_x, w8a_train_y)
p2 = m2.predict(w8a_test_x)
print("Accuracy:" + " "+str(accuracy_score(w8a_test_y, p2)))

plt.figure(figsize=(20,10))

plt.subplot(2,2,1)
plt.plot(np.array(range(n_feat)),m1.coef_, 'o-')
plt.title('Coefficients values')
plt.ylabel('I-th coeff')
plt.subplot(2,2,3)
plt.plot(np.array(range(num_it+1)), m1.l_f, 'r-')
plt.xlabel('Iterations')
plt.ylabel('Loss-Function')

plt.subplot(2,2,2)
plt.plot(np.array(range(n_feat)),m2.coef_, 'o-')
plt.title('Coefficients values')
plt.ylabel('I-th coeff')
plt.subplot(2,2,4)
plt.plot(np.array(range(num_it+1)), m2.l_f, 'r-')
plt.xlabel('Iterations')
plt.ylabel('Loss-Function')

plt.subplots_adjust

plt.show()

Comment the results

**Answer**:

In [ ]:
"Mini Batch - not avaraging"
"Dim 10"
dim_batch = 10
m31 = LogisticRegression(lam, dim_batch, False, 'l', 1e-2, num_it)
m31.fit(w8a_train_x, w8a_train_y)
p31 = m31.predict(w8a_test_x)
print("Accuracy:" + " "+str(accuracy_score(w8a_test_y, p31)))

"Mini Batch - not avaraging"
"Dim 20"
dim_batch = 20
m32 = LogisticRegression(lam, dim_batch, False, 'l', 1e-2, num_it)
m32.fit(w8a_train_x, w8a_train_y)
p32 = m32.predict(w8a_test_x)
print("Accuracy:" + " "+str(accuracy_score(w8a_test_y, p32)))

"Mini Batch - not avaraging"
"Dim 30"
dim_batch = 30
m33 = LogisticRegression(lam, dim_batch, False, 'l', 1e-2, num_it)
m33.fit(w8a_train_x, w8a_train_y)
p33 = m33.predict(w8a_test_x)
print("Accuracy:" + " "+str(accuracy_score(w8a_test_y, p33)))


plt.figure(figsize=(20,10))

plt.subplot(2,3,1)
plt.plot(np.array(range(n_feat)),m31.coef_, 'o-')
plt.title('Coefficients values')
plt.ylabel('I-th coeff')
plt.subplot(2,3,4)
plt.plot(np.array(range(num_it+1)), m31.l_f, 'r-')
plt.xlabel('Iterations')
plt.ylabel('Loss-Function')

plt.subplot(2,3,2)
plt.plot(np.array(range(n_feat)),m32.coef_, 'o-')
plt.title('Coefficients values')
plt.ylabel('I-th coeff')
plt.subplot(2,3,5)
plt.plot(np.array(range(num_it+1)), m32.l_f, 'r-')
plt.xlabel('Iterations')
plt.ylabel('Loss-Function')

plt.subplot(2,3,3)
plt.plot(np.array(range(n_feat)),m33.coef_, 'o-')
plt.title('Coefficients values')
plt.ylabel('I-th coeff')
plt.subplot(2,3,6)
plt.plot(np.array(range(num_it+1)), m33.l_f, 'r-')
plt.xlabel('Iterations')
plt.ylabel('Loss-Function')

plt.subplots_adjust

plt.show()

In [ ]:
-